In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
%cd /gdrive/My Drive/efficientnet

/gdrive/My Drive/efficientnet


In [0]:
!unzip data3.zip

In [0]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [0]:

# create the base pre-trained model
base_model = MobileNetV2(weights='imagenet',input_shape=(224,224,3), include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
#x = Dropout(0.5)(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.3)(x)
# and a logistic layer -- let's say we have 3 classes
predictions = Dense(4, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
print(len(base_model.layers))
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers[:100]:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=500,
    decay_rate=0.9)
optimizer = SGD(learning_rate=lr_schedule, momentum=0.9)
model.compile(optimizer=optimizer, loss='categorical_crossentropy')


model_file = "models/Epoch{epoch:02d}-loss{loss:.3f}-val{val_loss:.5f}.h5"
checkpoint = ModelCheckpoint(model_file, monitor='val_loss',verbose=1, save_best_only=True,mode='min')
callbacks_list = [checkpoint]

# model.summary()




9412608/9406464 [==============================] - 0s 0us/step
155


In [0]:
train_datagen = ImageDataGenerator(
        # rescale=1./255,
        shear_range=1,
        width_shift_range=10,
        rotation_range = 1,
        brightness_range=[0,2],
        validation_split=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        './data3',
        target_size=(224, 224),
        batch_size=12,
        shuffle=True,
        subset='training')
validation_generator = train_datagen.flow_from_directory(
        './data3',
        target_size=(224, 224),
        batch_size=12,
        subset='validation',
        shuffle=True)


Found 719 images belonging to 4 classes.
Found 179 images belonging to 4 classes.


In [0]:
model.fit(
        train_generator,
        steps_per_epoch = train_generator.samples // 12,
        epochs=120,
        validation_data = validation_generator,
        validation_steps = validation_generator.samples // 12,
        callbacks=callbacks_list)

In [0]:
# model.evaluate_generator(train_generator)
model.save("models/Epoch120-prune.h5")

In [0]:
from tensorflow.keras.preprocessing import image

from tensorflow.keras.models import load_model
import numpy as np

model = load_model('Epoch113-loss0.020-val0.00079-updated.h5')

In [0]:
import os
import time
direction = './data3/other/'
c = 0
d =os.listdir(direction)
for i in d:
    img_path = direction + i
    img = image.load_img(img_path,target_size=(224,224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x/255
    t1= time.time()
    y = model.predict(x)
    t2= time.time()
    print(t2-t1)
    cl = (np.argmax(y))
    print(y[0][cl])
    if np.argmax(y) != 2:
        c+=1

print(c,len(d))
print(100*(1-c/len(d)),'%')

#     x = tf.constant(x)
#     t1= time.time()
#     labeling = infer(x)
#     preds = labeling['dense_9'].numpy()
#     t2= time.time()
#     print(t2-t1)
#     idm = (np.argmax(preds))
#     print(preds[0][idm]*100)
#     if np.argmax(preds) != 1:
#         c+=1
# print(t2-t1)
# print(c,len(d))
# print(100*(1-c/len(d)),'%')

In [0]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 12
epochs = 2
# validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = 717
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.90,
                                                               final_sparsity=0.90,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer=model.optimizer, loss='categorical_crossentropy')

model_for_pruning.summary()

In [0]:
import tempfile
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]
  
model_for_pruning.fit(
        train_generator,
        steps_per_epoch=train_generator.samples//12,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples//12,
        callbacks=callbacks)

Epoch 1/10
 2/59 [>.............................] - ETA: 41s - loss: 1.1987WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.646390). Check your callbacks.


59/59 [==============================] - 19s 330ms/step - loss: 0.7130 - val_loss: 1.6878
Epoch 2/10
59/59 [==============================] - 17s 291ms/step - loss: 0.3626 - val_loss: 2.1026
Epoch 3/10
59/59 [==============================] - 17s 289ms/step - loss: 0.2323 - val_loss: 1.8908
Epoch 4/10
59/59 [==============================] - 17s 290ms/step - loss: 0.1937 - val_loss: 1.8393
Epoch 5/10
59/59 [==============================] - 17s 290ms/step - loss: 0.1704 - val_loss: 1.7047
Epoch 6/10
12/59 [=====>........................] - ETA: 10s - loss: 0.2065

KeyboardInterrupt: ignored

In [0]:
from tensorflow.keras.models import save_model
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: /tmp/tmpak6yx3gc.h5


In [0]:
from tensorflow.lite.python.lite import TFLiteConverterV2
converter = TFLiteConverterV2.from_keras_model(model)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

Saved pruned TFLite model to: /tmp/tmpnznlpwgt.tflite
